In [298]:
from tqdm import tqdm
import pandas as pd
from binance.client import Client

import numpy as np
import ta
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands, AverageTrueRange 
import re as re
from sqlalchemy.sql import exists  
from sqlalchemy import select
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker, relationship

In [299]:
client = Client()

In [300]:
from sqlalchemy import create_engine

pd.set_option("display.max_rows", 100, "display.max_columns", 100)

In [301]:
coins = ('BTCUSDT','ETHUSDT','BNBUSDT','SOLUSDT','ADAUSDT','XRPUSDT','DOTUSDT','LUNAUSDT',
  'DOGEUSDT','AVAXUSDT','SHIBUSDT','MATICUSDT','LTCUSDT','UNIUSDT','ALGOUSDT','TRXUSDT',
         'LINKUSDT','MANAUSDT','ATOMUSDT','VETUSDT')

In [302]:
fast = 7
slow = 25


In [303]:
def getminutedata(symbol, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol,'1m',lookback + ' days ago UTC' ))

    frame = frame.iloc[:,:6]
    frame.drop_duplicates()
    frame.columns = ['Time','Open','High','Low','Close','Volume']
    frame[['Open','High','Low','Close','Volume']] = frame[['Open','High','Low','Close','Volume']].astype(float)
    frame.Time = pd.to_datetime(frame.Time, unit='ms')
    return frame

In [304]:
engine = db.create_engine('sqlite:///Cryptoprices3.db')
#echo = True

In [305]:
Session = sessionmaker(bind=engine)
session = Session()

In [306]:
#users = session.query(Time)
#exists = db.session.query(User.id).filter_by(name='davidism').first() is not None

In [307]:
#

In [308]:
#for coin in tqdm(coins):
   #getminutedata(coin,'10').to_sql(coin, engine,if_exists="append", index=False)


In [309]:
test=pd.read_sql('BTCUSDT',engine).set_index('Time')

In [310]:
def groups_from_conditions(cond1, cond2):
    '''
    assign a unique non-NaN integer to each group as defined by the rules
    '''
    n = len(cond1)
    
    group_idx = -1
    groups = np.zeros(n)

    curr_state = 0    # 0 = not in a group, 1 = in a group
    for n in range(n):
        if curr_state == 0:
            # Currently not in a group
            if cond1[n] == 1:
                # Detected start of a group. so:
                # switch the state to 1 ie in a group
                curr_state = 1
                # get a new group_idx
                group_idx = group_idx + 1
                # assign it to the output for element n
                groups[n] = group_idx
            else:
                # no start of the group detected, we are not in a group so mark as NaN
                groups[n] = np.NaN

        else: 
            # current_state == 1 so we are in a group
            if cond2[n] == 1:
                # detected end of group -- switch state to 0
                curr_state = 0
            # as we are in a group assign current group_idx. Note that this happens for the element
            # for which cond2[n] == 1 as well, ie this element is included
            groups[n] = group_idx

    return groups

In [311]:

def getMax(df2):
    test_cond1 = df2['buy'].values
    test_cond2 = df2['sell'].values
    test_groups = groups_from_conditions(test_cond1, test_cond2)
    test_df = pd.DataFrame({'buy' : test_cond1, 'sell' : test_cond2, 'groups' : test_groups})
    max = test_df['groups'].max()
    return max

In [313]:
class Strat:
    
    def technicals(self, df):
        self.df = df
        df = df.copy()
        df = df.resample('5min').ffill()
        df.dropna(inplace=True)
        df['return'] = np.log(df.Close.pct_change()+ 1)
        df['SMA_fast'] = df.Close.rolling(7).mean()
        df['SMA_slow'] = df.Close.rolling(25).mean()
        df['position'] = np.where(df['SMA_fast'] > df['SMA_slow'],1,0)  # true or false = 1 eller 0
        df['strategyreturn'] = df['position'].shift(1)*df['return'] # when true enter next possible timestep
        df.dropna(inplace=True)
        return df


    def bb(self, df):
        self.df = df
        df = df.copy()
        df = df.resample('5min').ffill()
        df.dropna(inplace=True)
        bb_indicator = BollingerBands(df['Close'])
        df['return'] = np.log(df.Close.pct_change()+ 1)
        df['bb_high'] = bb_indicator.bollinger_hband()
        df['bb_low'] = bb_indicator.bollinger_lband()
        df['moving_avg'] = bb_indicator.bollinger_mavg()
        df['buy'] = np.where(df['Close'] < df['bb_low'], 1, 0).astype(float)
        df['sell'] = np.where(df['Close'] > df['bb_high'], 1, 0).astype(float)
        test_cond1 = df['buy'].values
        test_cond2 = df['sell'].values
        test_groups = groups_from_conditions(test_cond1, test_cond2)
        df['groups'] = test_groups
        df['position'] = np.where(df['groups'].isnull(),0,1)
        df['strategyreturn'] = df['position'].shift(1)*df['return'] # when true enter next possible timestep
        df.to_string("x")
        df.dropna(inplace=True)
        return df




In [314]:
def bb(df):
    df = df.copy()
    df = df.resample('5min').ffill()
    df.dropna(inplace=True)
    bb_indicator = BollingerBands(df['Close'])
    df['return'] = np.log(df.Close.pct_change()+ 1)
    df['bb_high'] = bb_indicator.bollinger_hband()
    df['bb_low'] = bb_indicator.bollinger_lband()
    df['moving_avg'] = bb_indicator.bollinger_mavg()
    df['buy'] = np.where(df['Close'] < df['bb_low'], 1, 0).astype(float)
    df['sell'] = np.where(df['Close'] > df['bb_high'], 1, 0).astype(float)
    test_cond1 = df['buy'].values
    test_cond2 = df['sell'].values
    test_groups = groups_from_conditions(test_cond1, test_cond2)
    df['groups'] = test_groups
    df['position'] = np.where(df['groups'].isnull(),0,1)
    df['strategyreturn'] = df['position'].shift(1)*df['return'] # when true enter next possible timestep
    df.to_string("x")
    df.dropna(inplace=True)
    return df

In [341]:
def rsistrat(df):
    df = df.copy()
    df = df.resample('5min').ffill()
    df.dropna(inplace=True)
    rsi_indicator = RSIIndicator(df['Close'])
    df = df.iloc[16: , :]
    df['return'] = np.log(df.Close.pct_change()+ 1)
    df['rsi'] = rsi_indicator.rsi()
    df['buy'] = np.where(df['rsi'] < 30, 1, 0).astype(float)
    df['sell'] = np.where(df['rsi'] > 70, 1, 0).astype(float)
    test_cond1 = df['buy'].values
    test_cond2 = df['sell'].values
    test_groups = groups_from_conditions(test_cond1, test_cond2)
    df['groups'] = test_groups
    df['position'] = np.where(df['groups'].isnull(),0,1)
    df['strategyreturn'] = df['position'].shift(1)*df['return'] # when true enter next possible timestep
    df.to_string('rsi')
    df.dropna(inplace=True)
    return df

In [368]:
def compare(df, func, func2):
    df = df.copy()
    df['true'] = func(df)['buy'] == 1 & func2(df)['buy'] == 1
    #df['compare'] = np.where(func(df)['buy'] == 1 | func2(df)['buy'] == 1, True, False)
    return df

In [369]:
compare(test,bb, rsistrat)

TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]

SyntaxError: invalid syntax (Temp/ipykernel_6640/3575393420.py, line 1)

In [ ]:
for coin in coins:
    df = pd.read_sql(coin, engine).set_index('Time')
    print(coin)
    trades = technicals(df).position.diff().value_counts().iloc[1:].sum()
    print(trades)
    costs = trades * 0.00075
    print(np.exp(technicals(df)['return'].sum())-1)
    print(np.exp(technicals(df)[['strategyreturn']].sum())-1-costs)

    #iloc[1: exclude the first value which is total number of trades]

BTCUSDT
10
-0.02572061501615852
strategyreturn   -0.018742
dtype: float64
ETHUSDT
10
-0.03613181398213816
strategyreturn   -0.004956
dtype: float64
BNBUSDT
16
-0.022410147991544394
strategyreturn   -0.034213
dtype: float64
SOLUSDT
10
-0.049156221553113166
strategyreturn   -0.013753
dtype: float64
ADAUSDT
10
-0.004691689008043287
strategyreturn    0.036398
dtype: float64
XRPUSDT
12
-0.026654411764705177
strategyreturn   -0.011606
dtype: float64
DOTUSDT
12
-0.06178489702517098
strategyreturn   -0.034291
dtype: float64
LUNAUSDT
14
-0.07237487733071646
strategyreturn   -0.036965
dtype: float64
DOGEUSDT
12
-0.03214285714285825
strategyreturn   -0.010614
dtype: float64
AVAXUSDT
12
-0.03949751018560399
strategyreturn   -0.016287
dtype: float64
SHIBUSDT
10
-0.041422160856058765
strategyreturn   -0.007738
dtype: float64
MATICUSDT
12
-0.07245744216499339
strategyreturn   -0.044641
dtype: float64
LTCUSDT
10
-0.07666886979511045
strategyreturn   -0.025109
dtype: float64
UNIUSDT
16
1.55431223447521

In [326]:
def printOut(func):
    for coin in coins:
        df = pd.read_sql(coin, engine).set_index('Time')
        print(coin)
        trades = bb(df)['groups'].max() + 1
        print(trades)
        costs = trades * 0.00075
        print(np.exp(func(df)['return'].sum())-1)
        print(np.exp(func(df)[['return','strategyreturn']].sum())-1-costs)

In [327]:
printOut(bb)

BTCUSDT
30.0
-0.03744721973035092
return           -0.059947
strategyreturn    0.015610
dtype: float64
ETHUSDT
28.0
-0.07166811331189782
return           -0.092668
strategyreturn   -0.011712
dtype: float64
BNBUSDT
29.0
-0.11556771825077361
return           -0.137318
strategyreturn   -0.058384
dtype: float64
SOLUSDT
33.0
-0.11271847710857696
return           -0.137468
strategyreturn    0.003944
dtype: float64
ADAUSDT
29.0
-0.06902527492273014
return           -0.090775
strategyreturn    0.064428
dtype: float64
XRPUSDT
28.0
-0.09746354913997868
return           -0.118464
strategyreturn   -0.036220
dtype: float64
DOTUSDT
34.0
-0.0767541258196568
return           -0.102254
strategyreturn    0.055545
dtype: float64
LUNAUSDT
30.0
-0.12986034537261104
return           -0.152360
strategyreturn   -0.016481
dtype: float64
DOGEUSDT
32.0
-0.12082090510607657
return           -0.144821
strategyreturn    0.018250
dtype: float64
AVAXUSDT
28.0
-0.1580468144448447
return           -0.179047
strategyret

In [ ]:
##df['EMA200'] = ta.trend.ema_indicator(df.Close, window=200)
#df['wf_Top_bool'] = np.where(df['High'].rolling(9, center=True).max(), True, False)

#0.075/100